In [1]:
import datetime
import hashlib
import re
import pandas as pd

from pathlib import Path

from orfdb import base, settings, annotation_loading
from orfdb.base import Assembly, Gene, Transcript, TranscriptExon, Exon, Orf, SequenceRegion, Dataset,\
                        Cds, CdsOrf, SequenceRegionXref, OrfXref,\
                        TranscriptOrf, Protein

from sqlalchemy import inspect, and_, update
from sqlalchemy_batch_inserts import enable_batch_inserting

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
session = base.Session()

In [3]:
t = session.query(Transcript).first()

In [4]:
t.transcript_idx_str

'127588411_127591700_+_8_154;80;109;71;125;487_127588411;127589083;127589485;127590066;127590963;127591213'

In [5]:
t2 = session.query(Transcript).filter(Transcript.refseq_id != '').first()

In [6]:
t2.transcript_idx_str

'127588411_127591700_+_8_154;80;109;71;125;487_127588411;127589083;127589485;127590066;127590963;127591213'

In [9]:
session.query(Transcript).filter(Transcript.transcript_idx_str == '138875400_138935034_+_6_197;93;52;92;80;152;92;156;200;151;110;181;105;134;1232_138875400;138880195;138880681;138881045;138886212;138887490;138904349;138917742;138924510;138925256;138929246;138930473;138930830;138932578;138933802').all()

[]

In [8]:
t2

Transcript: (1, ENST00000000233.10, NM_001662.4)+:127588411-127591700

In [16]:
session.query(Transcript).count()

444772